In [1]:
import pandas as pd
import numpy as np

In [2]:
import regex as re
from datetime import datetime


In [3]:
# traigo el dataset original de robados
data = pd.read_csv('./robo_autos/curados/recuperados.csv')

In [4]:
data.drop(['registro_seccional_codigo', 'automotor_tipo_codigo', 'automotor_marca_codigo','automotor_modelo_codigo', 'automotor_uso_codigo', 
          'titular_domicilio_provincia_indec_id', 'titular_pais_nacimiento_indec_id'], axis = 1, inplace = True)

--------------------

In [5]:
data.shape

(2992, 18)

In [8]:
#data.automotor_tipo_descripcion.unique()
data.columns

Index(['tramite_tipo', 'tramite_fecha', 'fecha_inscripcion_inicial',
       'registro_seccional_descripcion', 'registro_seccional_provincia',
       'automotor_origen', 'automotor_anio_modelo',
       'automotor_tipo_descripcion', 'automotor_marca_descripcion',
       'automotor_modelo_descripcion', 'automotor_uso_descripcion',
       'titular_tipo_persona', 'titular_domicilio_localidad',
       'titular_domicilio_provincia', 'titular_genero',
       'titular_anio_nacimiento', 'titular_pais_nacimiento',
       'titular_porcentaje_titularidad'],
      dtype='object')

In [8]:
data['automotor_tipo_descripcion'].nunique()

17

In [104]:
# Checkear los nombres que solo tienen el string...
#data[data['automotor_tipo_descripcion'].str.contains('FURGON')]['automotor_tipo_descripcion'].unique()
data[data['automotor_tipo_descripcion'].str.contains('SEDAN')]['automotor_tipo_descripcion'].unique()

array(['SEDAN 4 P', 'SEDAN 3 P', 'SEDAN'], dtype=object)

In [42]:
# los primeros 20 suman 60k
data.groupby('automotor_tipo_descripcion')['tramite_tipo'].count().sort_values(ascending= False)[0:20].sum()

57985

In [9]:
data.groupby('automotor_tipo_descripcion')['tramite_tipo'].count().sort_values(ascending= False)[0:40]

automotor_tipo_descripcion
SEDAN 4 P             2129
SEDAN 3 P              281
PICK UP                248
FURGON                 170
TODO TERRENO            63
COUPE                   39
FAMILIAR                18
TRACTOR                 10
CHASIS CON CABINA       10
ACOPLADO                 5
SEMIRREMOLQUE            5
UTILITARIO               4
SEDAN                    3
TRANSP. PASAJEROS        2
COMBI                    2
SIN ESPECIFICACION       2
CAMION                   1
Name: tramite_tipo, dtype: int64

In [206]:
# drop rows where tipo isunique?

## tips regex

    - '.' busca cualquier caracter
    - '*' busca cualquier numero de repeticiones de lo que está antes
    - '\s' busca cualquier whitespace


## DECIDÍ AGRUPAR MUCHO MÁS PORQUE SI NO VA A SER UN DELIRIO Y MILLONES DE DUMMIES, GENERALIZO MÁS (SEDAN, PICKUP, ETC)

In [86]:
#data['automotor_tipo_descripcion'].replace(to_replace= '^(?!SEDAN.3.P$).*$.*SEDAN.*', value= 'SEDAN 4 P', regex= True, inplace= True)
# ACCESORIO --> busca todo lo que NO SEA sedan 3.p y si tenga sedan
data['automotor_tipo_descripcion'].replace(to_replace= '^(?!.*SEDAN.3.P.*).*SEDAN.*', value= 'SEDAN 4 P', regex= True, inplace= True)

In [7]:
data['automotor_tipo_descripcion'].replace(to_replace= '.*SEDAN\s+4.*', value= 'SEDAN 4 P', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*S.*4.*', value= 'SEDAN 4 P', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*BELINA.*4.*', value= 'SEDAN 4 P', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*CUATRO.*PUERTAS.*| .*4.*PUERTAS.*', value= 'SEDAN 4 P', regex= True, inplace= True)

data['automotor_tipo_descripcion'].replace(to_replace= '.*SEDAN\s+5.*', value= 'SEDAN 4 P', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*SEDAN\s+3.*', value= 'SEDAN 3 P', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*3\s+P.*', value= 'SEDAN 3 P', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*5\s+P.*', value= 'SEDAN 4 P', regex= True, inplace= True)


# SEDAN 2 PUERTAS ES COUPE
data['automotor_tipo_descripcion'].replace(to_replace= '.*SEDAN\s+2.*', value= 'COUPE', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*2\s+P.*', value= 'COUPE', regex= True, inplace= True)

#TODO LO QUE QUEDA COMO SEDAN PASA A SER 4 PUERTAS
data['automotor_tipo_descripcion'].replace(to_replace= '^(?!.*SEDAN.3.P.*).*SEDAN.*', value= 'SEDAN 4 P', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= 'SEDAN$', value= 'SEDAN 4 P', regex= True, inplace= True)

#LO QUE QUEDA DE PUERTAS SON SEDANES
#data['automotor_tipo_descripcion'].replace(to_replace= '.*PUERTA.*', value= 'SEDAN', regex= True, inplace= True)
# SEDAN VA TODO JUNTO (NO, DIVIDIR POR PUERTAS)
#data['automotor_tipo_descripcion'].replace(to_replace= '.*S.DAN.*', value= 'SEDAN', regex= True, inplace= True)

#data['automotor_tipo_descripcion'].replace(to_replace= 'RURAL\s+4.*', value= 'RURAL 4 P', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*TRACTOR.*', value= 'TRACTOR', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= 'MINIBUS.*', value= 'MINIBUS', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*FAMILIAR.*|.*FAMLIAR.*', value= 'FAMILIAR', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= 'BERLINA\s+3.*', value= 'BERLINA 3 P', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= 'BERLINA\s+4.*', value= 'BERLINA 4 P', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= 'BERLINA\s+5.*', value= 'BERLINA 5 P', regex= True, inplace= True)
#data['automotor_tipo_descripcion'].replace(to_replace= '.*FURGON VID.*', value= 'FURGON VIDRIADO', regex= True, inplace= True)
# UNIFICAR TODO FURGON Y FURGONETA
data['automotor_tipo_descripcion'].replace(to_replace= '.*FURGON.*|.*FOUR.*', value= 'FURGON', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*SEMIRREMOLQUE.*', value= 'SEMIRREMOLQUE', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*REMOLQUE.*', value= 'SEMIRREMOLQUE', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*PICK.*UP.*', value= 'PICK UP', regex= True, inplace= True)
#RURAL PASA A SER FAMILIAR
data['automotor_tipo_descripcion'].replace(to_replace= '.*RURAL.*', value= 'FAMILIAR', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*4+X+4.*', value= 'TODO TERRENO', regex= True, inplace= True)
# FURGONETA PASA A SER FURGON
data['automotor_tipo_descripcion'].replace(to_replace= '.*FURGONETA.*', value= 'FURGON', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*FOURGON.*', value= 'FURGON', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*FUR.*', value= 'FURGON', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*FGON.*|.*FUR.*', value= 'FURGON', regex= True, inplace= True)

data['automotor_tipo_descripcion'].replace(to_replace= '.*CH.*', value= 'CHASIS CON CABINA', regex= True, inplace= True)
#data['automotor_tipo_descripcion'].replace(to_replace= '.*CH*', value= 'CHASIS CON CABINA', regex= True, inplace= True)
# SEDAN Y BERLINA VAN A TERMINAR TODOS JUNTOS
data['automotor_tipo_descripcion'].replace(to_replace= '.*BERLINA.*', value= 'SEDAN', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*COUPE.*', value= 'COUPE', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*DEPOR.*', value= 'COUPE', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*PASAJERO.*|.*PERSON.*', value= 'TRANSP. PASAJEROS', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*OMNIBUS.*', value= 'TRANSP. PASAJEROS', regex= True, inplace= True)
# TODO ESCOLAR PASA A SER TRANSP PASAJEROS
data['automotor_tipo_descripcion'].replace(to_replace= '.*ESC.*', value= 'TRANSP. PASAJEROS', regex= True, inplace= True)

# BREAK ES UN FAMILIAR, GUIENSÁ
data['automotor_tipo_descripcion'].replace(to_replace= '.*BREAK.*', value= 'FAMILIAR', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*OMBI.*', value= 'COMBI', regex= True, inplace= True)
# TODO LO QUE SEA BUS/VAN PASA A SER COMBI
data['automotor_tipo_descripcion'].replace(to_replace= '.*BUS.*', value= 'COMBI', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*VAN.*', value= 'COMBI', regex= True, inplace= True)

#data['automotor_tipo_descripcion'].replace(to_replace= '.*CAMION/s.*', value= 'CAMION', regex= True, inplace= True)
# PASAR CAMIONETA A PICKUP PRIMERO PARA EVITAR LLEVARMELA PUESTA CON CAMION (IGUALMENTE VER COMO EXLCUIR)
data['automotor_tipo_descripcion'].replace(to_replace= '.*CAMIONETA.*', value= 'PICK UP', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*CAMION.*', value= 'CAMION', regex= True, inplace= True)
# TODO LO QUE TENGA CARGA ES CAMBION
data['automotor_tipo_descripcion'].replace(to_replace= '.*CARGA.*', value= 'CAMION', regex= True, inplace= True)

# CONVERTIBLE PASA A SER DESCAPOTABLE
data['automotor_tipo_descripcion'].replace(to_replace= '.*CONV.*', value= 'DESCAPOTABLE', regex= True, inplace= True)
# TODO LO QUE QUEDA CON PTAS ES SEDAN
data['automotor_tipo_descripcion'].replace(to_replace= '.*PTAS.*', value= 'SEDAN', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*WAGON.*', value= 'FAMILIAR', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*UTIL.*', value= 'UTILITARIO', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*AUTO.*', value= 'SEDAN', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*JEEP.*', value= 'TODO TERRENO', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*AMB.*', value= 'AMBULANCIA', regex= True, inplace= True)

data['automotor_tipo_descripcion'].replace(to_replace= '.*ACOPL.*', value= 'ACOPLADO', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*CASA.*', value= 'CASA RODANTE', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*HOME.*', value= 'CASA RODANTE', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '.*GRUA.*', value= 'GRUA', regex= True, inplace= True)


In [109]:
#PROTOCOLO 21 SON AUTOS FABRICADOS EN EL MERCOUSR

In [13]:
# remover los tipos que tienen menos de 2 ocurrencias
conteo = data['automotor_tipo_descripcion'].value_counts() < 2
remover = conteo[conteo].index
data = data[~data.automotor_tipo_descripcion.isin(remover)]
# remover SIN ESPECIFICACION
data = data[data['automotor_tipo_descripcion'] != 'SIN ESPECIFICACION']

In [970]:
data['automotor_tipo_descripcion'].value_counts()

SEDAN 4 P            41421
SEDAN 3 P             7540
PICK UP               4885
FURGON                4376
COUPE                 1804
TODO TERRENO          1008
CHASIS CON CABINA      614
FAMILIAR               572
TRACTOR                223
SEDAN                  216
SEMIRREMOLQUE          159
TRANSP. PASAJEROS      155
CAMION                 107
UTILITARIO             100
COMBI                   91
ACOPLADO                34
AMBULANCIA               9
OTROS                    8
ARENERO                  5
GRUA                     4
DESCAPOTABLE             4
CASA RODANTE             3
COMERCIAL LIVIANO        2
LIMUSINA                 2
Name: automotor_tipo_descripcion, dtype: int64

In [14]:
data.shape

(61259, 18)

## MARCAS

In [975]:
data.groupby('automotor_marca_descripcion')['tramite_tipo'].count().sort_values(ascending= False)[0:66].sum()

63252

In [10]:
data['automotor_marca_descripcion'].nunique()

52

In [368]:
data[data['automotor_marca_descripcion'].str.contains('RINO')]['automotor_marca_descripcion'].unique()

array(['TORINO'], dtype=object)

In [18]:
data['automotor_marca_descripcion'].value_counts()[40:67]

SMART                11
RANDON               10
LIFAN                10
BONANO               10
AST-PRA               9
MONTENEGRO            9
OPEL                  8
PINCEN                8
EL DETALLE            8
DACIA                 8
RASTROJERO DIESEL     7
PETINARI              7
MIC                   7
LADA                  6
CORMETAL              6
DS                    5
PORSCHE               5
TRENQUEMOLQUE         4
GOMATRO               4
TORINO                4
SOLA Y BRUSA          4
GAMMA                 3
ASIA                  3
INNOCENTI             3
A.F.F.                3
DFM                   3
SANXING               2
Name: automotor_marca_descripcion, dtype: int64

In [11]:
data['automotor_marca_descripcion'].replace(to_replace= 'W.*N$|V.*N$|.*VOLKSWAGEN.*|.*VW.*', value= 'VOLKSWAGEN', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '^R.*T$|.*RENAULT.*|.*RENA.*', value= 'RENAULT', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*FIAT.*|.*FIA.*', value= 'FIAT', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*FORD.*', value= 'FORD', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*GMC.*|.*MOT.*|^C.*T$|.*CHEV.*', value= 'CHEVROLET', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*PEUGE.*', value= 'PEUGEOT', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*CITR.*', value= 'CITROEN', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*TOY.*', value= 'TOYOTA', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*BEN.*', value= 'MERCEDES BENZ', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*SUZ.*|.*UKI.*', value= 'SUZUKI', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*NIS.*', value= 'NISSAN', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '^B.*W', value= 'BMW', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*JEEP.*', value= 'JEEP', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*KIA.*', value= 'KIA', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*MIT.*', value= 'MITSUBISHI', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*ALF.*', value= 'ALFA ROMEO', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*DODGE.*|RAM$', value= 'DODGE', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*SEAT.*', value= 'SEAT', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*ROV.*', value= 'LAND ROVER', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*SCANIA.*', value= 'SCANIA', regex= True, inplace= True)
data['automotor_marca_descripcion'].replace(to_replace= '.*AGRA.*', value= 'DEUTZ AGRALE', regex= True, inplace= True)

# EXCEPCIONES


In [19]:
# remover los tipos que tienen menos de 3 ocurrencias
conteo2 = data['automotor_marca_descripcion'].value_counts() <= 3
remover2 = conteo2[conteo2].index
data = data[~data.automotor_marca_descripcion.isin(remover2)]


## MODELO?

In [939]:
data.groupby('automotor_modelo_descripcion')['tramite_tipo'].count().sort_values(ascending= False)[0:185].sum()

58905

In [14]:
data['automotor_modelo_descripcion'].nunique()

291

In [940]:
data[data['automotor_modelo_descripcion'].str.contains('156')]['automotor_modelo_descripcion'].unique()

array(['156 2.0TS', '156 2.4 JTD', '11569', '156 2.0 TS',
       '508 ALLURE N 156 CV TIP', '156 2.5 V6 24V'], dtype=object)

In [15]:
data['automotor_modelo_descripcion'].value_counts()[180:220]

IMPREZA 1.6 4D GL 5S            1
XV                              1
LANOS 1.5 S                     1
ONE COUNTRYMAN                  1
330 D                           1
NISSAN PATHFINDER TD27 5M/T     1
SENTRA TEKNA XTRONIC CVT        1
PB F2 HD 12,80 M.               1
SENTRA SENSE PURE DRIVE         1
KOLEOS 2.5 EXPRESSION           1
PREMIUM 380 DXI                 1
CRONOS                          1
320D                            1
C 200                           1
IBIZA                           1
KICKS EXCLUSIVE CVT F2          1
X-TERRA SE                      1
VITO 111 CDI                    1
450 E 31 T                      1
450E37T                         1
CORONA 2.0                      1
SFPLA 15,5 2+1E                 1
SCIROCCO 1.4 TSI DSG            1
L 1215/48                       1
CABINA DOBLE 4X4 2.5 TDI        1
M 3                             1
SORRENTO EX V6                  1
LAND CRUISER PRADO VX A/T       1
KOLEOS DYNAMIQUE 4X4 MT         1
X1 20I        

In [13]:
data['automotor_modelo_descripcion'].replace(to_replace= '.*GOL\s.*', value= 'GOL', regex= True, inplace= True)
data['automotor_tipo_descripcion'].replace(to_replace= '^(?!.*GOLF.*).*GOL.*', value= 'GOL', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*GOLF\s.*', value= 'GOLF', regex= True, inplace= True)

data['automotor_modelo_descripcion'].replace(to_replace= '.*RENAULT\s+9.*|.*RENAULT9.*|^9\s.*|.*R-9.*', value= 'RENAULT 9', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*RENAULT\s+19.*|.*RENAULT19.*|^19\s.*|.*R-19.*', value= 'RENAULT 19', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*RENAULT\s+11.*|.*RENAULT-11.*|^11\s.*|.*R-11.*', value= 'RENAULT 11', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*RENAULT\s+12.*|.*RENAULT-12.*|^12\s.*|.*R-12.*', value= 'RENAULT 12', regex= True, inplace= True)

data['automotor_modelo_descripcion'].replace(to_replace= '.*FIORINO.*', value= 'FIORINO', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*FOX.*', value= 'FOX', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*CORSA.*', value= 'CORSA', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*UNO.*', value= 'UNO', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '^KA\s.*', value= 'KA', regex= True, inplace= True)
# 147 Y TODOS SUS DERIVADOS
data['automotor_modelo_descripcion'].replace(to_replace= '.*SPAZ.*|.*VIVACE.*|^147.*', value= '147', regex= True, inplace= True)

data['automotor_modelo_descripcion'].replace(to_replace= '.*VOY.*', value= 'VOYAGE', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '^CLASSIC.*', value= 'CLASSIC', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '^ECO.*', value= 'ECOSPORT', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '^ESC.*', value= 'ESCORT', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*BORA.*', value= 'BORA', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '^MERI.*', value= 'MERIVA', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*SEND.*', value= 'SENDA', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*F.*?100.*', value= 'F-100', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*F.*?150.*', value= 'F-150', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*F.*?350.*', value= 'F-350', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*ASTRA.*', value= 'ASTRA', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*SIENA.*', value= 'SIENA', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*VENTO.*', value= 'VENTO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*DUNA.*', value= 'DUNA', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '^FUN.*', value= 'FUN', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*PART.*|.*PARP.*|^PAR.*', value= 'PARTNER', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*PALIO.*', value= 'PALIO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*FIESTA.*', value= 'FIESTA', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*TWINGO.*', value= 'TWINGO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*105\s+.*|^105.*|.*\s+105.*', value= '105', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*106\s+.*|^106.*|.*\s+106.*', value= '106', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*205\s+.*|^205.*|.*\s+205.*', value= '205', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*206\s+.*|^206.*|.*\s+206.*', value= '206', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*207\s+.*|^207.*|.*\s+207.*', value= '207', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*208\s+.*|^208.*|.*\s+208.*', value= '208', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*2008\s+.*|^2008.*|.*\s+2008.*', value= '2008', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*305\s+.*|^305.*|.*\s+305.*', value= '305', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*306\s+.*|^306.*|.*\s+306.*', value= '306', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*307\s+.*|^307.*|.*\s+307.*', value= '307', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*308\s+.*|^308.*|.*\s+308.*', value= '308', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*405\s+.*|^405.*|.*\s+405.*', value= '405', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*406\s+.*|^406.*|.*\s+406.*', value= '406', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*407\s+.*|^407.*|.*\s+407.*', value= '407', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*408\s+.*|^408.*|.*\s+408.*', value= '408', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*504\s+.*|^504.*|.*\s+504.*', value= '504', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*505\s+.*|^505.*|.*\s+505.*', value= '505', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)

data['automotor_modelo_descripcion'].replace(to_replace= '.*POLO.*', value= 'POLO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*C3\s+.*', value= 'C3', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*BERLINGO.*|^BER.*', value= 'BERLINGO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*ONIX.*', value= 'ONIX', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*FOCUS.*', value= 'FOCUS', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*TIGGO.*', value= 'TIGGO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*POINTER.*', value= 'POINTER', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*AVEO.*', value= 'AVEO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*CLIO.*', value= 'CLIO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*ZAFIRA.*', value= 'ZAFIRA', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*PUNTO.*', value= 'PUNTO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*AGILE.*', value= 'AGILE', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*KANGOO.*', value= 'KANGOO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*HILUX.*', value= 'HILUX', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*SURAN.*', value= 'SURAN', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '^EXPRES.*', value= 'EXPRESS', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*VECTRA.*', value= 'VECTRA', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*COURIER.*', value= 'COURIER', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*CELTA.*', value= 'CELTA', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*FIT.*', value= 'FIT', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*ORION.*', value= 'ORION', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*RANGER.*', value= 'RANGER', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*SAVEIRO.*', value= 'SAVEIRO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*SANDERO.*', value= 'SANDERO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*COROL.*', value= 'COROLLA', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*MEGAN.*', value= 'MEGANE', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*CIVIC.*', value= 'CIVIC', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*S10.*', value= 'S10', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*PRISMA.*', value= 'PRISMA', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*LOGAN.*', value= 'LOGAN', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '^HIGH.*', value= 'HIGH', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*SARA.*|.*PICAS.*|.*XSARA.*', value= 'XSARA', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*MOVE.*', value= 'MOVE', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*CRUZE.*', value= 'CRUZE', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*ETIOS.*', value= 'ETIOS', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*SPIN\s+.*', value= 'SPIN', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*C.*?RV.*', value= 'CRV', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*H.*?RV.*', value= 'HRV', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*REGATTA.*|.*REGAT.*', value= 'REGATTA', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*CADD.*', value= 'CADDY', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*EXPLOR.*', value= 'EXPLORER', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*TRACK.*', value= 'TRACKER', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*MOBI.*', value= 'MOBI', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '^PT.*', value= 'PT CRUISER', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*FALCON.*', value= 'FALCON', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*C3\s+.*', value= 'C3', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*C4\s+.*', value= 'C4', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*C5\s+.*', value= 'C5', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*SONIC.*', value= 'SONIC', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*PASSAT.*', value= 'PASSAT', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*BOXER.*', value= 'BOXER', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*TRAF.*', value= 'TRAFIC', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '^500\s+.*', value= '500', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '^CARG.*', value= 'CARGO', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*MONDEO.*', value= '500', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*SPRINT.*', value= 'SPRINTER', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*STRADA.*', value= 'STRADA', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*NEON.*', value= 'NEON', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*BLAZER.*', value= 'BLAZER', regex= True, inplace= True)
data['automotor_modelo_descripcion'].replace(to_replace= '.*TRANSIT.*', value= 'TRANSIT', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*JOURNEY.*', value= 'JOURNEY', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*TAKE.*', value= 'TAKE', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '^ARGO.*', value= 'ARGO', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*FLUENCE.*', value= 'FLUENCE', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*TIIDA.*', value= 'TIIDA', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '^A4.*', value= 'A4', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '^A5.*', value= 'A5', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*DS3.*', value= 'DS3', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*SORENTO.*', value= 'SORENTO', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*ATOS.*', value= 'ATOS', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '^LS.*', value= 'LS', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '^146.*', value= '146', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*MASTER.*', value= 'MASTER', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*Q5.*', value= 'Q5', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*AMAROK.*', value= 'AMAROK', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*ACCORD.*', value= 'ACCORD', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*MB.*', value= 'MB', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*IDEA.*', value= 'IDEA', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*GACEL.*', value= 'GACEL', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '^A3.*', value= 'A3', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*SW4.*', value= 'SW4', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*L160.*', value= 'L160', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '^3008.*', value= '3008', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*RENEGADE.*', value= 'RENEGADE', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*CRONO.*', value= 'CRONOS', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*COBALT.*', value= 'COBALT', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*DUSTER.*', value= 'DUSTER', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*A160.*', value= 'A160', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*4000.*', value= 'F4000', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*SWIFT.*', value= 'SIWFT', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*GALAXY.*', value= 'GALAXY', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*TIPO.*', value= 'TIPO', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*TORO.*', value= 'TORO', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*CAPTIVA.*', value= 'CAPTIVA', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*CARAVAN.*', value= 'CARAVAN', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*SHARAN.*', value= 'SHARAN', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*SANTAMO.*', value= 'SANTAMO', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*ACCENT.*', value= 'ACCENT', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*V.*1500.*|^1500.*', value= '1500', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*DURAMO.*', value= 'DURAMO', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*SANTA FE.*', value= 'SANTA FE', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*CHEROKEE.*', value= 'CHEROKEE', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*323.*', value= '323', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*C15.*', value= 'C15', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*RAM.*', value= 'RAM', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*QQ.*', value= 'QQ', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*FRONT.*', value= 'FRONTIER', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*STILO.*', value= 'STILO', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*DUCATO.*', value= 'DUCATO', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '^120.*', value= '120', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*416.*', value= '416', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*145.*', value= '145', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*SCENIC.*', value= 'SCENIC', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*TAUNUS.*', value= 'TAUNUS', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*LANCER.*', value= 'LANCER', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*KUGA.*', value= 'KUGA', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*SPARK.*', value= 'SPARK', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*AXOR.*', value= 'AXOR', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*SANTANA.*', value= 'SANTANA', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*A1\s+.*', value= 'A1', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*DOBLO.*', value= 'DOBLO', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*KWID.*', value= 'KWID', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*LAGUNA.*', value= 'LAGUNA', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*CITY.*', value= 'CITY', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*COOPER.*', value= 'COOPER', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*TIGUAN.*', value= 'TIGUAN', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*CAPTUR.*', value= 'CAPTUR', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*BEETLE.*', value= 'BEETLE', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*B2500.*', value= 'B2500', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*B200.*', value= 'B200', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*LEON.*', value= 'LEON', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*LINEA.*', value= 'LINEA', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*COMPASS.*', value= 'COMPASS', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*MONTANA.*', value= 'MONTANA', regex= True, inplace= True) 
data['automotor_modelo_descripcion'].replace(to_replace= '.*C.*?200.*', value= 'C 200', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*C.*?220.*', value= 'C 220', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*C.*?250.*', value= 'C 250', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*MUSSO.*', value= 'MUSSO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*DAILY.*', value= 'DAILY', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '^214.*', value= '214', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '^216.*', value= '216', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '^600.*', value= '600', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*ATEGO.*', value= 'ATEGO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*X5.*', value= 'X5', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '^318.*', value= '318', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*528.*', value= '528', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*CORDOBA.*', value= 'CORDOBA', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*SAXO.*', value= 'SAXO', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*IBIZA.*', value= 'IBIZA', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*FULWIN.*', value= 'FULWIN', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*INCA.*', value= 'INCA', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '.*420.*', value= '420', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '^H1.*', value= 'H1', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)
data['automotor_modelo_descripcion'].replace(to_replace= '^156.*', value= '156', regex= True, inplace= True) #(NOTHING OR MORE BETWEEN)



In [24]:
conteo3 = data['automotor_modelo_descripcion'].value_counts() < 15
remover3 = conteo3[conteo3].index
data = data[~data.automotor_modelo_descripcion.isin(remover3)]

In [16]:
data.shape

(2992, 18)

In [17]:
data.drop_duplicates(inplace= True)

In [18]:
data.shape

(2989, 18)

In [19]:
data.isnull().sum()

tramite_tipo                       0
tramite_fecha                      0
fecha_inscripcion_inicial          0
registro_seccional_descripcion     0
registro_seccional_provincia       0
automotor_origen                   0
automotor_anio_modelo              5
automotor_tipo_descripcion         0
automotor_marca_descripcion        0
automotor_modelo_descripcion       0
automotor_uso_descripcion          0
titular_tipo_persona               0
titular_domicilio_localidad        6
titular_domicilio_provincia        0
titular_genero                     0
titular_anio_nacimiento           27
titular_pais_nacimiento            0
titular_porcentaje_titularidad     0
dtype: int64

In [20]:
data.dropna(inplace= True)

In [21]:
data.shape

(2953, 18)

In [22]:
data.to_csv('C:/Users/ilcarbo/Documents/DATA/backup cagadas/recus_reg.csv', index= False)

In [23]:
robos = pd.read_csv('C:/Users/ilcarbo/Documents/DATA/backup cagadas/robos_reg.csv')

In [24]:
EDA = pd.concat([robos, data], ignore_index= True)

In [25]:
EDA.shape

(60673, 18)

In [26]:
EDA.head(3)

,tramite_tipo,tramite_fecha,fecha_inscripcion_inicial,registro_seccional_descripcion,registro_seccional_provincia,automotor_origen,automotor_anio_modelo,automotor_tipo_descripcion,automotor_marca_descripcion,automotor_modelo_descripcion,automotor_uso_descripcion,titular_tipo_persona,titular_domicilio_localidad,titular_domicilio_provincia,titular_genero,titular_anio_nacimiento,titular_pais_nacimiento,titular_porcentaje_titularidad
0,DENUNCIA DE ROBO O HURTO,2018-01-17,2000-10-05,ESTEBAN ECHEVERRIA Nº 1,Buenos Aires,Nacional,2000.0,SEDAN 4 P,CHEVROLET,CORSA,Privado,Física,MONTE GRANDE,BUENOS AIRES,Masculino,1981.0,Argentina,100
1,DENUNCIA DE ROBO O HURTO,2018-01-03,2007-11-22,LANUS Nº 1,Buenos Aires,Nacional,2007.0,FURGON,PEUGEOT,PARTNER,Privado,Física,VALENTIN ALSINA,BUENOS AIRES,Femenino,1990.0,Argentina,100
2,DENUNCIA DE ROBO O HURTO,2018-01-12,1995-02-01,MAR DEL PLATA Nº 02,Buenos Aires,Nacional,1995.0,SEDAN 4 P,RENAULT,RENAULT 19,Privado,Física,UNIDAD TURISTICA CHAPADMALAL,BUENOS AIRES,Masculino,1986.0,Argentina,100


In [33]:
EDA['titular_pais_nacimiento'] = EDA['titular_pais_nacimiento'].map(lambda x: x.upper())

In [29]:
EDA['titular_pais_nacimiento'] = EDA['titular_pais_nacimiento'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [34]:
# Tratar casos especiales y cambiar (z franca cdo rivadavia y rivera)
EDA['titular_pais_nacimiento'] = EDA['titular_pais_nacimiento'].replace('Z.F. CDRO.RIVADAVIA(ARG.)', 'ARGENTINA')
EDA['titular_pais_nacimiento'] = EDA['titular_pais_nacimiento'].replace('Z.FRANCA CDRO. RIVADAVIA (ARG)', 'ARGENTINA')
EDA['titular_pais_nacimiento'] = EDA['titular_pais_nacimiento'].replace('Z.F. PUNTA ARENAS (CHILE)', 'CHILE')
EDA['titular_pais_nacimiento'] = EDA['titular_pais_nacimiento'].replace('RIVERA (URUGUAY)', 'URUGUAY')
# Cambiar "NO APLICA" y "NO IDENTIFICADO" por NaN // REEMPLAZAR POR NO IDENTIFICADO? SI HACER
#todo['titular_pais_nacimiento'] = todo['titular_pais_nacimiento'].replace('NO APLICA', np.nan)
#todo['titular_pais_nacimiento'] = todo['titular_pais_nacimiento'].replace('NO IDENTIFICADO', np.nan)
EDA['titular_pais_nacimiento'] = EDA['titular_pais_nacimiento'].replace('NO APLICA', 'NO IDENTIFICADO')

In [35]:
cond_index = EDA[EDA['automotor_anio_modelo'] == 20].index
EDA.drop(cond_index, inplace = True)

In [36]:
EDA.shape

(60670, 18)

In [38]:
data.drop_duplicates(inplace= True)

In [39]:
EDA.shape

(60670, 18)

In [37]:
EDA.to_csv('C:/Users/ilcarbo/Documents/DATA/backup cagadas/para_EDA.csv', index= False)